# TSFS12 Hand-in exercise 3: Path following for autonomous vehicles

In [2]:
%pip install seaborn

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 KB 7.6 MB/s eta 0:00:008.2 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [3]:
import numpy as np
import matplotlib.pyplot as plt
from vehiclecontrol import ControllerBase, SingleTrackModel, PurePursuitControllerBase
from splinepath import SplinePath
from scipy.linalg import solve_discrete_are
from seaborn import despine

In [5]:
# Run if you want plots in external windows
%matplotlib  

Using matplotlib backend: TkAgg


In [3]:
# Run the ipython magic below to activate automated import of modules. Useful if you write code in external .py files.
# %load_ext autoreload
# %autoreload 2

# Make a simple controller and simulate vehicle

In [11]:
class MiniController(ControllerBase):
    def __init__(self):
        super().__init__()
    
    def u(self, t, w):
        a = 0.0
        if t < 10:
            u = [np.pi / 180 * 10, a]
        elif 10 <= t < 20:
            u = [-np.pi / 180 * 11, a]
        elif 20 <= t < 23:
            u = [-np.pi / 180 * 0, a]
        elif 23 <= t < 40:
            u = [-np.pi / 180 * 15, a]
        else:
            u = [-np.pi / 180 * 0, a]
        return u

opts = {'L': 2, 
        'amax': np.inf,
        'amin': -np.inf,
        'steer_limit': np.pi / 3}

car = SingleTrackModel().set_attributes(opts)
car.Ts = 0.1
car.controller = MiniController()
w0 = [0, 0, 0, 2]
z0 = car.simulate(w0, T=40, dt=0.1, t0=0.0)
t, w, u = z0
M = 10
p = w[::M, 0:2]
nom_path = SplinePath(p)

In [5]:
# s = np.linspace(0, nom_path.length, 100)

# _, ax = plt.subplots(num=10, clear=True)
# ax.plot(nom_path.x(s), nom_path.y(s))
# ax.plot(p[:, 0], p[:, 1], 'rx')
# ax.set_xlabel('x [m]')
# ax.set_ylabel('y [m]')
# ax.set_title('Path from simple controller')
# ax.axis('square')

# fig, ax = plt.subplots(1, 2, num=11, clear=True, layout='constrained')
# ax[0].plot(t, u[:, 0]*180/np.pi)
# ax[0].set_xlabel('t [s]')
# ax[0].set_ylabel('steer [deg]')
# ax[0].set_title('Steer')

# ax[1].plot(t, u[:, 1])
# ax[1].set_xlabel('t [s]')
# ax[1].set_ylabel('acceleration [m/s^2]')
# ax[1].set_title('Acceleration')

# Pure pursuit controller

In [12]:
class PurePursuitController(PurePursuitControllerBase):
    def __init__(self, l, L, path=None, goal_tol=1., pursuit_point_fig=None):
        """
        Create pure-pursuit controller object
        
        Input arguments:
          l -- Prediction horizon
          L -- wheel-base
          path -- SplinePath-object with the path to follow
          goal_tol -- Goal stopping tolerance (default: 1)
          pursuit_point_fig -- Figure object to illustrate pursuit-point selection (default: None)
        """
        super().__init__(pursuit_point_fig)
        self.plan = path
        self.l = l
        self.L = L
        self.goal_tol = goal_tol

        
    def pursuit_point(self, p_car):
        """ Return pure-pursuit given the position of the car.
        
          Input:
            p_car - Car position in global coordinates
            
          Return:
            pursuit point in global coordinates
        """
        # p_car - position of vehicle

        path_points = self.plan.path  # Points on the path
        l = self.l  # Pure-pursuit look-ahead

        close_points = []
        for point in path_points:
            d2 = np.linalg.norm(p_car - point)
            if d2 < l:
                close_points.append(point)
        p_purepursuit = close_points[-1]
    
        # Your code here

        # Hint: It is typically not important to find a point at _exactly_ distance l, 
        #       for example search pure-pursuit point among the points in path_points
        #       but don't forget to take into account the approximate pursuit-horizon when
        #       computing the steering angle.
        # p_purepursuit = [0, 0]
        return p_purepursuit
    
    def pure_pursuit_control(self, dp, theta):
        """ Compute pure-pursuit steer angle.
        
          Input:
            dp - Vector from position of car to pursuit point
            theta - heading of vehicle
          
          Output:
            return steer angle
        """

        l = np.linalg.norm(dp)
        rot_mat = np.array([[np.cos(theta - np.pi/2), -np.sin(theta - np.pi/2)],
                            [np.sin(theta - np.pi/2),  np.cos(theta - np.pi/2)]])
        h_x = np.matmul(rot_mat, (np.array([[1, 0]]).reshape(-1, 1)))
        x = np.dot(dp, h_x)
        delta = np.arctan2(-self.L*2*x/(l**2), 1)

        return delta[0]

    def u(self, t, w):
      
        """ Compute control action
        
          Input:
            t - current time
            w - current state w = (x, y, theta, v)
          
          Output:
            return (delta, acc) where delta is steer angle and acc acceleration
        """
        x, y, theta, v = w
        p_car = np.array([x, y])

        # Your code here to compute steering angle, use the functions
        # self.pursuit_point() and self.pure_pursuit_control() you 
        # have written above.

        p_purepursuit = self.pursuit_point(p_car)
        dp = p_purepursuit - p_car
        delta = self.pure_pursuit_control(dp, theta)
        
        acc = 0
        self._pursuit_plot(p_car, p_purepursuit)  # Included for animation purposes.
        return np.array([delta, acc])
    
    def run(self, t, w):
        # Function that returns true until goal is reached
        p_goal = self.plan.path[-1, :]
        p_car = w[0:2]
        dp = p_car - p_goal
        dist = dp.dot(dp)        
        
        return dist > self.goal_tol**2

## Assertions

A few tests on your implementation. Note that passing these tests doesn't imply that your solution is correct but do not submit a solution if your solution doesn't pass these tests. First, test the ```pure_pursuit_control``` function

In [13]:
pp_controller = PurePursuitController(l=4, L=car.L, path=nom_path, goal_tol=0.25)
assert abs(pp_controller.pure_pursuit_control(np.array([1., 1.]), 10 * np.pi / 180) - 1.01840) < 1e-3
assert abs(pp_controller.pure_pursuit_control(np.array([1., 1.]), 30 * np.pi / 180) - 0.6319) < 1e-3
assert abs(pp_controller.pure_pursuit_control(np.array([1., 1.]), -30 * np.pi / 180) - 1.2199) < 1e-3

To ensure that the pursuit-point selection works, run a simulation with pure-pursuit illustration turned on. Requires plotting in external window.

In [9]:
%matplotlib

Using matplotlib backend: TkAgg


In [10]:
s = np.linspace(0, nom_path.length, 200)
fig, ax = plt.subplots(num=99, clear=True)
ax.plot(nom_path.x(s), nom_path.y(s), 'b', lw=0.5)
ax.plot(nom_path.path[:, 0], nom_path.path[:, 1], 'rx', markersize=3)

car = SingleTrackModel()
car.controller = PurePursuitController(l=5, L=car.L, path=nom_path, goal_tol=0.25, pursuit_point_fig=fig)

w0 = [0, 1, np.pi/2 * 0.9, 2]
# z_pp = car.simulate(w0, T=80, dt=0.1, t0=0.0)

## Simulate controller

Simulate controller without visualization

In [15]:
s = np.linspace(0, nom_path.length, 200)
fig, ax = plt.subplots(num=99, clear=True)
ax.plot(nom_path.x(s), nom_path.y(s), 'b', lw=0.5)
ax.plot(nom_path.path[:, 0], nom_path.path[:, 1], 'rx', markersize=3)

car = SingleTrackModel()
car.controller = PurePursuitController(l=4, L=car.L, path=nom_path)

w0 = [0, 1, np.pi / 2 * 0.9, 2]  # Sample starting state
z_pp = car.simulate(w0, T=80, dt=0.1, t0=0.0)
t_pp, w_pp, u_pp = z_pp
M = 2
p_pp = w_pp[::M, 0:2]
ax.plot(p_pp[:, 0], p_pp[:, 1])
despine()

Plot path error

In [16]:
d_pp = nom_path.path_error(w_pp[:, :2])
fig, ax = plt.subplots(num=100, clear=True)
ax.plot(t_pp, d_pp)
ax.set_title("Path distance error")
ax.set_ylabel("Error [m]")
ax.set_xlabel("Time [s]")

Text(0.5, 0, 'Time [s]')

# State feedback controller based on the linearized path

Implement linear and non-linear state feedback control.

In [23]:
class StateFeedbackController(ControllerBase):
    def __init__(self, K, L, path=None, goal_tol=1.):
        super().__init__()
        self.plan = path
        self.K = K
        self.goal_tol = goal_tol
        self.d = []
        self.L = L
        self.s0 = 0
        self.u_prev = 0

    def heading_error(self, theta, s):
        """Compute theta error
        Inputs
            theta - current heading angle
            s - projection point on path
            
        Outputs
            theta_e - heading error angle
        """

        # get heading of vehicle, hv
        rot_mat = np.array([[np.cos(theta), -np.sin(theta)],
                            [np.sin(theta),  np.cos(theta)]])
        hv = np.matmul(rot_mat, (np.array([[1, 0]]).reshape(-1, 1)))
        hv = np.squeeze(hv)

        # get heading of path, hs
        hs, _ = self.plan.heading(s)

        # get angle between hv, hs
        cos_theta_e = np.dot(hs, hv)/(np.linalg.norm(hs) * np.linalg.norm(hv))
        hv_ext = np.hstack((hv, 0))
        hs_ext = np.hstack((hs, 0))

        cross = np.cross(hs_ext, hv_ext)/(np.linalg.norm(hs) * np.linalg.norm(hv))
        sin_theta_e = cross[2]
    
        theta_e = np.arctan2(sin_theta_e, cos_theta_e)
        return theta_e

    def u(self, t, w):
        x, y, theta, v = w
        p_car = w[0:2]

        # Compute d and theta_e errors. Use the SplinePath method project
        # and the obj.heading_error() function you've written above

        s_project, d = self.plan.project(p_car, self.s0) # d should be signed!
        self.s0 = s_project
        theta_e = self.heading_error(theta, s_project)

        # car.Ts = timestep size (sample period)

        # the feed-forward term is supposed to be the curvature of the path, I think.
        # k_u = 0.7
        # u = k_u * self.u_prev - self.K[0]*d - self.K[1]*theta_e
        u = self.plan.c(s_project) - self.K[0]*d - self.K[1]*theta_e
        self.u_prev = u

        

        # Compute control signal delta
        delta = np.arctan2(u*self.L, 1)
        acc = 0  # Constant speed

        return np.array([delta, acc])
    
    def run(self, t, w):
        p_goal = self.plan.path[-1, :]
        p_car = w[0:2]
        dp = p_car - p_goal
        dist = np.sqrt(dp.dot(dp))
        
        return dist > self.goal_tol**2


### Simulate and visualize

Calculate feedback gain K

In [24]:
v = 2  # linearize around v = 2
Ts = car.Ts
A = np.array([[1, Ts*v],
              [0, 1]])
B = np.array([[0],
              [Ts]])
Q = np.array([[20, 0],
              [0, 30]])
R = np.array([5])
P = solve_discrete_are(A, B, Q, R)
# K = (R+(B^T)*P*B)^-1 * (B^T)*P*A
K = np.squeeze(1/(R + np.transpose(B) @ P @ B) @ np.transpose(B) @ P @ A)

print(K)

[1.6590092  3.45102302]


In [19]:
%matplotlib

Using matplotlib backend: TkAgg


In [25]:
s = np.linspace(0, nom_path.length, 200)
fig, ax = plt.subplots(num=99, clear=True)
ax.plot(nom_path.x(s), nom_path.y(s), 'b', lw=0.5)
ax.plot(nom_path.path[:, 0], nom_path.path[:, 1], 'rx', markersize=3)

car = SingleTrackModel()
car.controller = StateFeedbackController(K=K, L=car.L, path=nom_path, goal_tol=0.45)

# w0 = [0, 1, np.pi / 2 * 0.9, v]
# w0 = [-5, 10, 0.9 * np.pi / 2, v]
z_fb = car.simulate(w0, T=80, dt=0.1, t0=0.0)

t_fb, w_fb, u_fb = z_fb
M = 2
p_fb = w_fb[::M, 0:2]
ax.plot(p_fb[:, 0], p_fb[:, 1])
despine()

Plot path error

In [16]:
d_fb = nom_path.path_error(w_fb[:, :2])
fig, ax = plt.subplots(num=100, clear=True)
ax.plot(t_fb, d_fb)
ax.set_title("Path distance error")
ax.set_ylabel("Error [m]")
ax.set_xlabel("Time [s]")


Text(0.5, 0, 'Time [s]')

# Non-linear State feedback controller

In [26]:
class NLStateFeedbackController(ControllerBase):
    def __init__(self, K, L, path=None, goal_tol=1.):
        super().__init__()
        self.plan = path
        self.K = K
        self.goal_tol = goal_tol
        self.d = []
        self.L = L
        self.s0 = 0
        self.u_prev = 0

    def heading_error(self, theta, s):
        """Compute theta error
        Inputs
            theta - current heading angle
            s - projection point on path
            
        Outputs
            theta_e - heading error angle
        """

        # get heading of vehicle, hv
        rot_mat = np.array([[np.cos(theta), -np.sin(theta)],
                            [np.sin(theta),  np.cos(theta)]])
        hv = np.matmul(rot_mat, (np.array([[1, 0]]).reshape(-1, 1)))
        hv = np.squeeze(hv)

        # get heading of path, hs
        hs, _ = self.plan.heading(s)

        # get angle between hv, hs
        cos_theta_e = np.dot(hs, hv)/(np.linalg.norm(hs) * np.linalg.norm(hv))
        hv_ext = np.hstack((hv, 0))
        hs_ext = np.hstack((hs, 0))

        cross = np.cross(hs_ext, hv_ext)/(np.linalg.norm(hs) * np.linalg.norm(hv))
        sin_theta_e = cross[2]
    
        theta_e = np.arctan2(sin_theta_e, cos_theta_e)
        return theta_e

    def u(self, t, w):
        x, y, theta, v = w
        p_car = w[0:2]

        # Compute d and theta_e errors. Use the SplinePath method project
        # and the obj.heading_error() function you've written above

        s_project, d = self.plan.project(p_car, self.s0) # d should be signed!
        self.s0 = s_project
        theta_e = self.heading_error(theta, s_project)

        # car.Ts = timestep size (sample period)

        # non-linear:
        m = 1 - (theta_e**2)/6 + (theta_e**4)/120 - (theta_e**6)/5040
        u = self.plan.c(s_project) - self.K[0]*m*d - self.K[1]*theta_e
        self.u_prev = u

        delta = np.arctan2(u*self.L, 1)

        acc = 0  # Constant speed

        return np.array([delta, acc])
    
    def run(self, t, w):
        p_goal = self.plan.path[-1, :]
        p_car = w[0:2]
        dp = p_car - p_goal
        dist = np.sqrt(dp.dot(dp))
        
        return dist > self.goal_tol**2


In [122]:
%matplotlib

Using matplotlib backend: TkAgg


In [27]:
s = np.linspace(0, nom_path.length, 200)
fig, ax = plt.subplots(num=99, clear=True)
ax.plot(nom_path.x(s), nom_path.y(s), 'b', lw=0.5)
ax.plot(nom_path.path[:, 0], nom_path.path[:, 1], 'rx', markersize=3)

car = SingleTrackModel()
car.controller = NLStateFeedbackController(K=K, L=car.L, path=nom_path, goal_tol=0.45)

# w0 = [0, 1, 0.9 * np.pi/2, 2]
# w0 = [-5, 10, 0.9 * np.pi / 2, v]
z_nl= car.simulate(w0, T=80, dt=0.1, t0=0.0)

t_nl, w_nl, u_nl = z_nl
M = 2
p_nl = w_nl[::M, 0:2]
ax.plot(p_nl[:, 0], p_nl[:, 1])
despine()

Plot path error

In [40]:
d_nl = nom_path.path_error(w_nl[:, :2])
fig, ax = plt.subplots(num=101, clear=True)
ax.plot(t_nl, d_nl, label="Non-linear state feedback")
ax.set_title("Path distance error")
ax.set_ylabel("Error [m]")
ax.set_xlabel("Time [s]")
ax.legend()

### Compare controllers

Modify K for comparisons

In [28]:
v = 2  # linearize around v = 2
Ts = car.Ts
A = np.array([[1, Ts*v],
              [0, 1]])
B = np.array([[0],
              [Ts]])
Q = np.array([[30, 0],
              [0, 20]])
R = np.array([1])
P = solve_discrete_are(A, B, Q, R)
# K = (R+(B^T)*P*B)^-1 * (B^T)*P*A
K = np.squeeze(1/(R + np.transpose(B) @ P @ B) @ np.transpose(B) @ P @ A)

print(K)

[3.96745177 5.54659917]


Plot paths

In [29]:
s = np.linspace(0, nom_path.length, 200)
fig, ax = plt.subplots(num=99, clear=True)
ax.plot(nom_path.x(s), nom_path.y(s), 'b', lw=0.5)
ax.plot(nom_path.path[:, 0], nom_path.path[:, 1], 'rx', markersize=3)

car_pp = SingleTrackModel()
car_fb = SingleTrackModel()
car_nl = SingleTrackModel()
car_pp.controller = PurePursuitController(l=4, L=car.L, path=nom_path)
car_fb.controller = StateFeedbackController(K=K, L=car.L, path=nom_path, goal_tol=0.60)
car_nl.controller = NLStateFeedbackController(K=K, L=car.L, path=nom_path, goal_tol=0.60)

w0 = [1, 2, 0.9 * np.pi/2, v]
z_pp= car_pp.simulate(w0, T=80, dt=0.1, t0=0.0)
z_fb= car_fb.simulate(w0, T=80, dt=0.1, t0=0.0)
z_nl= car_nl.simulate(w0, T=80, dt=0.1, t0=0.0)

t_pp, w_pp, u_pp = z_pp
t_fb, w_fb, u_fb = z_fb
t_nl, w_nl, u_nl = z_nl

M = 1
p_pp = w_pp[::M, 0:2]
p_fb = w_fb[::M, 0:2]
p_nl = w_nl[::M, 0:2]
ax.plot(p_pp[:, 0], p_pp[:, 1], label="Pure pursuit")
ax.plot(p_fb[:, 0], p_fb[:, 1], label="Linear state feedback")
ax.plot(p_nl[:, 0], p_nl[:, 1], label="Non-linear state feedback")
ax.legend()
despine()

Plot path error

In [124]:
d_pp = nom_path.path_error(w_pp[:, :2])
d_fb = nom_path.path_error(w_fb[:, :2])
d_nl = nom_path.path_error(w_nl[:, :2])
fig, ax = plt.subplots(num=101, clear=True)
ax.plot(t_pp, d_pp, label="Pure pursuit")
ax.plot(t_fb, d_fb, label="Linear state feedback")
ax.plot(t_nl, d_nl, label="Non-linear state feedback")
ax.set_title("Path distance error")
ax.set_ylabel("Error [m]")
ax.set_xlabel("Time [s]")
ax.legend()

Plot control signals

In [148]:
fig, ax = plt.subplots(num=102, clear=True)
# ax.plot(t_pp, u_pp[:,0], label="Pure pursuit")
ax.plot(t_fb, u_fb[:,0], label="Linear state feedback")
ax.plot(t_nl, u_nl[:,0], label="Non-linear state feedback")
ax.set_xlim(0, 5)
ax.set_title("Control input $\delta$")
ax.set_ylabel("Steering input [radians]")
ax.set_xlabel("Time [s]")
ax.legend()